In [10]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer
import math

import os

In [30]:
def angleBetweenPoints(p0, p1, p2):
    a = (p1[0] - p0[0]) ** 2 + (p1[1] - p0[1]) ** 2
    b = (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2
    c = (p2[0] - p0[0]) ** 2 + (p2[1] - p0[1]) ** 2
    return math.acos((a + b - c) / math.sqrt(4 * a * b))

class Agent:
    def __init__(self):
        self.featureExtractors = [getattr(self, m) for m in dir(self) if
                                  m.startswith("_Agent__extractFeature") and callable(getattr(self, m))]
        self.__reward=0
        self.__realStateDim = 82
        self.__stateDim = 6 + 8 * 6

    def __extractFeatureReward(self, *args):
        "Reward"
        return self.__reward

    def __extractFeatureCenterOfGravity(self, *args):
        "Center of gravity (equal masses of points)"
        state = args[0].copy()
        xy = state[0, 2:].reshape(int((self.__realStateDim - 2) / 4), 4)[:, :2]
        return np.sqrt(np.square(xy).sum(1)).mean()

    def __extractFeatureDistance(self, *args):
        "Euklidean distance from closest to (9,-1)"
        state = args[0].copy()
        xy = state[0, 2:].reshape(int((self.__realStateDim - 2) / 4), 4)[:,
             :2]  # lista punktów (x,y)(czubki segmentów macki)
        xy -= [9, -1]
        return np.sqrt(np.square(xy).sum(1).min())

    def __extractFeatureAngleParallelity(self, *args):
        state = args[0][0]
        "Angle stopping tentacle from parallelity with [(0,0),(9,-1)]"
        return angleBetweenPoints(state[34:36], (9, -1), state[38:40])

    def __extractFeatureAngleParallelity2(self, *args):
        state = args[0][0]
        "Angle stopping tentacle from parallelity with [(0,0),(9,-1)]"
        return angleBetweenPoints(state[74:76], (9, -1), state[78:80])

    def __extractFeatureVertexCloser(self, *args):
        "Which vertex of the tentacle is closer (-1 if lower, 1 if upper)"
        state = args[0].copy()
        xy = state[0, 2:].reshape(int((self.__realStateDim - 2) / 4), 4)[[9, 19],
             :2]  # lista punktów (x,y)(czubki segmentów macki)
        return np.argmin(np.square(xy - [9, -1]).sum(1)) * 2 - 1
    
    def __extractImportantPoints(self, *args):
        state = args[0][2:].copy()
        print(state)
        l = []
        for i in (0, 4, 5, 7, 8, 9):
            l+=(state[4 * i:4 * i + 4])
            l+=(state[4 * i + 40:4 * i + 44])
        for i in range(12):
            l[i*4] -= 4.5
            l[i*4+1] -= 1
        return l

    def getFeatureVector(self, state, reward):
        "Convert input parameters to vecture of features"
        self.__reward += reward
        st=state.copy()
        state = np.array(list(state)).reshape((1, self.__realStateDim))
        f = []
        for m in self.featureExtractors:
            s=state.copy()
            # print(m.__name__,m(state,reward))
            f += [m(s, reward)]
        f+=self.__extractImportantPoints(st)
        return f

In [31]:
ag = Agent()

In [35]:
model = Sequential()
model.add(InputLayer(batch_input_shape=(1, 6 + 8 * 6)))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(6 * 6 * 6, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

TypeError: 'int' object is not iterable

In [33]:
data = []
a = []
reward = []
f = open("data001", 'r')
for i,line in enumerate(f):
    if i%3==0:
        data.append([float(x) for x in line.split(", ")])
    elif i%3==1:
        a.append([int(float(x)) for x in line.split(", ")])
    else:
        reward.append(float(line))

In [34]:
f=[]
anwser=[]
for i in range(len(data)):
    f.append(ag.getFeatureVector(data[i], reward[i]))
for i in range(len(a)):
    anwser.append(np.zeros((6*6*6)))
    x=a[i]
    w=0
    print(x)
    if x[1]==0:w+=3
    else: w+=5
    w*=6
    if x[19]==0:w+=3
    else: w+=5
    w*=6
    if x[28]==0:w+=3
    else: w+=5
    anwser[i][w] = 9 + 0.04*i
    print(w)
     
    

[0.9735005146973044, 1.021735404805655, -0.004487761759996865, 0.013535664666621318, 1.9986941893857613, 1.0040097845034655, -0.0009988938308323103, -0.0016771959466422952, 3.0022982287328412, 1.0015141506716545, 0.0014290133238283919, -0.0036057839902006485, 4.0012915706088865, 0.9998448459510588, 0.0007239083817972544, -0.004887049762076894, 4.999957937631293, 1.0003315114895313, -5.5092695986828385e-05, -0.005201187054413256, 5.99616455006271, 1.004795901417845, -0.0024245223014218364, -0.004973108796880163, 6.987427476770761, 1.0143237428631038, -0.0076635111074865505, -0.004436184666302155, 7.974002425878227, 1.0250391304674114, -0.012647194445004934, 0.00040435707561409587, 8.948682697443855, 1.0677082937347409, -0.014454526699670069, 0.011829224605838705, 10.046937431450853, 0.951320143988048, 0.00017921717032010932, -0.026905282967584432, 1.0742879529465712, -0.0010506680263928222, 0.037519818698121356, 0.003076148927465383, 2.0200315028618063, -0.0013056566196183756, 0.0126337

In [27]:
x = []
a=215
for i in range(3):
    y = a % 6
    for j in range((5, 4, 1)[i]):
        x += {0: [0, 0, 0], 1: [1, 0, 0], 2: [0, 1, 0], 3: [0, 0, 1], 4: [1, 1, 0], 5: [0, 1, 1]}[y]
    a //= 6
print(x)

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]


In [28]:
batch_size = 1   # ile obrazków przetwarzamy na raz (aktualizacja wag sieci następuje raz na całą grupę obrazków)
epochs = 500         # ile epok będziemy uczyli

xx=np.array(f)
yy=np.array(anwser)
# Tutaj następuje właściwe uczenie
model.fit(xx, yy,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xx, yy))

Train on 11 samples, validate on 11 samples
Epoch 1/500
11/11 [==============================] - 0s 26ms/step - loss: 0.3978 - mean_absolute_error: 0.1620 - val_loss: 0.3775 - val_mean_absolute_error: 0.1194
Epoch 2/500
11/11 [==============================] - 0s 2ms/step - loss: 0.3678 - mean_absolute_error: 0.0989 - val_loss: 0.3563 - val_mean_absolute_error: 0.0804
Epoch 3/500
11/11 [==============================] - 0s 2ms/step - loss: 0.3499 - mean_absolute_error: 0.0726 - val_loss: 0.3429 - val_mean_absolute_error: 0.0654
Epoch 4/500
11/11 [==============================] - 0s 3ms/step - loss: 0.3373 - mean_absolute_error: 0.0620 - val_loss: 0.3307 - val_mean_absolute_error: 0.0573
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: 0.3255 - mean_absolute_error: 0.0541 - val_loss: 0.3193 - val_mean_absolute_error: 0.0510
Epoch 6/500
11/11 [==============================] - 0s 2ms/step - loss: 0.3143 - mean_absolute_error: 0.0489 - val_loss: 0.3088 - val_mean_

Epoch 51/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1583 - mean_absolute_error: 0.0375 - val_loss: 0.1580 - val_mean_absolute_error: 0.0371
Epoch 52/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1582 - mean_absolute_error: 0.0374 - val_loss: 0.1579 - val_mean_absolute_error: 0.0378
Epoch 53/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1579 - mean_absolute_error: 0.0377 - val_loss: 0.1577 - val_mean_absolute_error: 0.0373
Epoch 54/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1586 - mean_absolute_error: 0.0372 - val_loss: 0.1574 - val_mean_absolute_error: 0.0372
Epoch 55/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1582 - mean_absolute_error: 0.0372 - val_loss: 0.1572 - val_mean_absolute_error: 0.0381
Epoch 56/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1574 - mean_absolute_error: 0.0374 - val_loss: 0.1572 - val_mean_absolute_error: 0.0375
Epoch 57/500
11/

Epoch 101/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1394 - mean_absolute_error: 0.0387 - val_loss: 0.1386 - val_mean_absolute_error: 0.0400
Epoch 102/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1387 - mean_absolute_error: 0.0387 - val_loss: 0.1373 - val_mean_absolute_error: 0.0382
Epoch 103/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1371 - mean_absolute_error: 0.0380 - val_loss: 0.1361 - val_mean_absolute_error: 0.0369
Epoch 104/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1361 - mean_absolute_error: 0.0376 - val_loss: 0.1348 - val_mean_absolute_error: 0.0369
Epoch 105/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1344 - mean_absolute_error: 0.0370 - val_loss: 0.1335 - val_mean_absolute_error: 0.0369
Epoch 106/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1338 - mean_absolute_error: 0.0366 - val_loss: 0.1324 - val_mean_absolute_error: 0.0357
Epoch 107/

Epoch 151/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0914 - mean_absolute_error: 0.0272 - val_loss: 0.0907 - val_mean_absolute_error: 0.0267
Epoch 152/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0908 - mean_absolute_error: 0.0267 - val_loss: 0.0900 - val_mean_absolute_error: 0.0271
Epoch 153/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0902 - mean_absolute_error: 0.0272 - val_loss: 0.0894 - val_mean_absolute_error: 0.0271
Epoch 154/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0893 - mean_absolute_error: 0.0268 - val_loss: 0.0887 - val_mean_absolute_error: 0.0286
Epoch 155/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0888 - mean_absolute_error: 0.0284 - val_loss: 0.0880 - val_mean_absolute_error: 0.0291
Epoch 156/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0882 - mean_absolute_error: 0.0288 - val_loss: 0.0872 - val_mean_absolute_error: 0.0275
Epoch 157/

Epoch 201/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0592 - mean_absolute_error: 0.0240 - val_loss: 0.0587 - val_mean_absolute_error: 0.0213
Epoch 202/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0589 - mean_absolute_error: 0.0217 - val_loss: 0.0581 - val_mean_absolute_error: 0.0215
Epoch 203/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0589 - mean_absolute_error: 0.0221 - val_loss: 0.0578 - val_mean_absolute_error: 0.0231
Epoch 204/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0584 - mean_absolute_error: 0.0228 - val_loss: 0.0570 - val_mean_absolute_error: 0.0212
Epoch 205/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0571 - mean_absolute_error: 0.0217 - val_loss: 0.0564 - val_mean_absolute_error: 0.0202
Epoch 206/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0572 - mean_absolute_error: 0.0209 - val_loss: 0.0559 - val_mean_absolute_error: 0.0198
Epoch 207/

...

Epoch 300/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0185 - mean_absolute_error: 0.0124 - val_loss: 0.0181 - val_mean_absolute_error: 0.0116
Epoch 301/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0182 - mean_absolute_error: 0.0122 - val_loss: 0.0179 - val_mean_absolute_error: 0.0116
Epoch 302/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0180 - mean_absolute_error: 0.0114 - val_loss: 0.0177 - val_mean_absolute_error: 0.0110
Epoch 303/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0186 - mean_absolute_error: 0.0114 - val_loss: 0.0175 - val_mean_absolute_error: 0.0117
Epoch 304/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0183 - mean_absolute_error: 0.0121 - val_loss: 0.0172 - val_mean_absolute_error: 0.0111
Epoch 305/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0175 - mean_absolute_error: 0.0120 - val_loss: 0.0170 - val_mean_absolute_error: 0.0109
Epoch 306/

11/11 [==============================] - 0s 2ms/step - loss: 0.0095 - mean_absolute_error: 0.0077 - val_loss: 0.0092 - val_mean_absolute_error: 0.0074
Epoch 350/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0099 - mean_absolute_error: 0.0076 - val_loss: 0.0091 - val_mean_absolute_error: 0.0074
Epoch 351/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0092 - mean_absolute_error: 0.0078 - val_loss: 0.0089 - val_mean_absolute_error: 0.0073
Epoch 352/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0095 - mean_absolute_error: 0.0075 - val_loss: 0.0089 - val_mean_absolute_error: 0.0075
Epoch 353/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0089 - mean_absolute_error: 0.0073 - val_loss: 0.0087 - val_mean_absolute_error: 0.0069
Epoch 354/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0099 - mean_absolute_error: 0.0075 - val_loss: 0.0090 - val_mean_absolute_error: 0.0078
Epoch 355/500
11/11 [===

Epoch 399/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_absolute_error: 0.0058 - val_loss: 0.0046 - val_mean_absolute_error: 0.0057
Epoch 400/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_absolute_error: 0.0059 - val_loss: 0.0045 - val_mean_absolute_error: 0.0056
Epoch 401/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0048 - mean_absolute_error: 0.0059 - val_loss: 0.0045 - val_mean_absolute_error: 0.0055
Epoch 402/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0049 - mean_absolute_error: 0.0057 - val_loss: 0.0044 - val_mean_absolute_error: 0.0053
Epoch 403/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_absolute_error: 0.0060 - val_loss: 0.0044 - val_mean_absolute_error: 0.0052
Epoch 404/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0052 - mean_absolute_error: 0.0059 - val_loss: 0.0044 - val_mean_absolute_error: 0.0055
Epoch 405/

11/11 [==============================] - 0s 2ms/step - loss: 0.0028 - mean_absolute_error: 0.0049 - val_loss: 0.0026 - val_mean_absolute_error: 0.0049
Epoch 449/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0029 - mean_absolute_error: 0.0044 - val_loss: 0.0027 - val_mean_absolute_error: 0.0044
Epoch 450/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0028 - mean_absolute_error: 0.0044 - val_loss: 0.0026 - val_mean_absolute_error: 0.0041
Epoch 451/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0027 - mean_absolute_error: 0.0044 - val_loss: 0.0025 - val_mean_absolute_error: 0.0040
Epoch 452/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0027 - mean_absolute_error: 0.0041 - val_loss: 0.0025 - val_mean_absolute_error: 0.0042
Epoch 453/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0026 - mean_absolute_error: 0.0045 - val_loss: 0.0025 - val_mean_absolute_error: 0.0039
Epoch 454/500
11/11 [===

Epoch 498/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0021 - mean_absolute_error: 0.0055 - val_loss: 0.0018 - val_mean_absolute_error: 0.0045
Epoch 499/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0018 - mean_absolute_error: 0.0041 - val_loss: 0.0017 - val_mean_absolute_error: 0.0038
Epoch 500/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0018 - mean_absolute_error: 0.0038 - val_loss: 0.0017 - val_mean_absolute_error: 0.0037


In [36]:
x=f[0]
print(len(x))
# model.save("model")

54
